In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from pydantic import BaseModel
class TestStructuredOutput(BaseModel):
    a: int
    b: str
    

In [ ]:
def get_weather(location: str):
    """Get current temperature for a given location."""
    raise ValueError("connection issue. Please try again later.")  # Simulating an error for demonstration
    # return f"The current temperature in {location} is 72°F."

In [ ]:
from pydantic import BaseModel
class TestWeatherOutput(BaseModel):
    location: str
    temperature: str

In [ ]:
import os
from neo.mcp.client import MCPClient

module_dir = os.path.dirname(__import__("neo").__file__)
example_dir = os.path.join(module_dir, "../../examples")
mcp_client = MCPClient(
    name="demo",
    server_script_path=f"{example_dir}/mcp/mcp-server-demo/main.py",
)

# OpenAI

In [ ]:

from neo.models.providers.openai import OpenAIResponseModel
from neo.tools.internal_tools.openai import WebSearch

model = OpenAIResponseModel(model="gpt-4o", 
                           instruction="talk like a german shepherd",
                           timeaware=True,
                           boolean_response=False, 
                           structured_response_model=None,
                           tools=[WebSearch(), get_weather],
                           mcp_clients=[mcp_client],
                           )
test = await model.acreate("what time is it in los angeles?", return_response_object=False)  

In [ ]:
test.display()

In [ ]:
await model.acreate("what was the weather there", base_thread=test)


In [ ]:
test.display()

In [ ]:
await model.acreate(user_input=None, base_thread=test)

In [ ]:
test.display()

In [ ]:
await model.acreate(user_input='search the internet', base_thread=test)

In [ ]:
test.display()

# OpenAI Complete

In [ ]:
from neo.models.providers.openai import OpenAICompleteModel
model = OpenAICompleteModel(model="gpt-4o", instruction="talk like a german shepherd",
                           boolean_response=False, 
                           structured_response_model=None)
test = await model.acreate("hi")

In [ ]:
test.display()

In [ ]:
await model.acreate(user_input='what did i say?', base_thread=test)

In [ ]:
test.display()

# Image

In [ ]:
from neo.models.providers.openai import OpenAIResponseModel

model = OpenAIResponseModel(model="gpt-4.1", 
                           instruction="talk like a german shepherd",
                           )

In [ ]:
from neo.contexts import Context
from neo.types import contents as t

image_url = "https://upload.wikimedia.org/wikipedia/commons/thumb/d/dd/Gfp-wisconsin-madison-the-nature-boardwalk.jpg/2560px-Gfp-wisconsin-madison-the-nature-boardwalk.jpg"

context = Context(
    contents=[
        t.TextContent(data="What's in this image?"),
        t.ImageContent(data=image_url),
    ]
)

In [ ]:
test = await model.acreate(context, return_response_object=False)  


In [ ]:
test.display()

In [ ]:
await model.acreate("make a poem about this image", base_thread=test)  
test.display()

# Audio

In [ ]:
audio_url = "https://cdn.openai.com/API/docs/audio/alloy.wav"

## Native

In [ ]:
from neo.models.providers.openai import OpenAICompleteModel
model = OpenAICompleteModel(model="gpt-4o-audio-preview", instruction="talk like a german shepherd")

In [ ]:
from neo.contexts import Context
from neo.types import contents as t

context = Context(
    contents=[
        t.TextContent(data="What's in this recording?"),
        t.AudioContent(data=audio_url),
    ]
)
test = await model.acreate(context, return_response_object=False)
test.display()

In [ ]:
await model.acreate("what is the content of the conversation regardless of speaker IDs", base_thread=test)


In [ ]:
test.display()

## Transcribe

In [ ]:
audio_url = "https://cdn.openai.com/API/docs/audio/alloy.wav"

In [ ]:
from neo.models.providers.openai import OpenAIResponseModel
from neo.contexts import Context
from neo.types import contents as C
from neo.tools.speech.openai import aspeech2text
from neo.tools import Tool


context = Context(
    contents=[
        C.TextContent(data="What's in this recording?"),
        C.AudioTextContent(data=audio_url, transcription_handler=Tool(func=aspeech2text,)),
    ]
)

model = OpenAIResponseModel(model="gpt-4o")

thread = await model.acreate(context, return_response_object=False)

In [ ]:
thread.display()

# Document

In [ ]:
txt_url = "https://assets.anthropic.com/m/1cd9d098ac3e6467/original/Claude-3-Model-Card-October-Addendum.pdf"

In [ ]:
from neo.models.providers.openai import OpenAIResponseModel
model = OpenAIResponseModel(model="gpt-4o", instruction="talk like a german shepherd")

from neo.contexts import Context
from neo.types import contents as t


# context = Context(
#     contents=[
#         t.DocumentContent(
#             data=txt_url, 
#             ),
#         t.TextContent(data="Summarize this document."),
#     ]
# )

context = Context(
    contents=[
        t.DocumentTextContent( # Text only mode; 5x faster and cheaper
            data=txt_url, 
            ),
        t.TextContent(data="Summarize this document."),
    ]
)

In [ ]:
test = await model.acreate(user_input=context)
test.display()

In [ ]:
context.contents[0].text